# Collection person locations within frame and building a csv dataset

### Notebook 3 

In this notebook I will take the progress in using the model to identify pedestrians within frame and build a datat set of pedestrian location within a single frame.

By logging the coordinates within the following frame I'll attempt to link that coordinates that have moved very slightly will likely be from the pedestrian in the previous frame.

From this I will build a csv data set to use for future location predicition

In [22]:
import os
import sys
import numpy as np
np.random.seed(0)
import matplotlib.pyplot as plt
import cv2
import json
import re

%matplotlib inline 

# Root directory of the project
ROOT_DIR = os.path.abspath("../")
print("Root Directory: ", ROOT_DIR)
sys.path.append(ROOT_DIR)

from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
from mrcnn.model import log

from scripts import coco
from scripts import list_file_info as lfi
from scripts import video_to_frames as vtf
from scripts.centroid_tracker import CentroidTracker
from scripts.trackable_object import TrackableObject

Root Directory:  /home/sam/Documents/GitHub/honoursProject/PedestrianDetectionSGall


In [12]:
data_root = "/home/sam/Desktop/sam_bdd_videos/videos_for_frames/"

file_list = os.listdir(path = data_root) 
max = len(file_list) 

if max == 0:
    print("No video folders in directory")
else:
    print("please select video from 1 to " + str(max))

please select video from 1 to 4


In [13]:
# Select a video 

selected_video = 4

vid_dir = "/home/sam/Desktop/sam_bdd_videos/videos_for_frames/" + str(selected_video) + "/video/"
frames_dir = "/home/sam/Desktop/sam_bdd_videos/videos_for_frames/" + str(selected_video) + "/frames/"
final_vid_dir = "/home/sam/Desktop/sam_bdd_videos/videos_for_frames/" + str(selected_video) + "/output_vid/"
output_dir = "/home/sam/Desktop/sam_bdd_videos/videos_for_frames/" + str(selected_video) + "/output_img/"
json_output_dir = "/home/sam/Desktop/sam_bdd_videos/videos_for_frames/" + str(selected_video) + "/output_json/"

In [14]:
class InferenceConfig(coco.CocoConfig):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

# Device to load the neural network on.
# Useful if you're training a model on the same 
# machine, in which case use CPU and leave the
# GPU for training.
DEVICE = "/gpu:0"  # /cpu:0 or /gpu:0

In [15]:
# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "weights/mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

class_names = ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
                'bus', 'train', 'truck', 'boat', 'traffic light',
                'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
                'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
                'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
                'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
                'kite', 'baseball bat', 'baseball glove', 'skateboard',
                'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
                'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
                'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
                'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
                'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
                'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
                'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
                'teddy bear', 'hair drier', 'toothbrush']

config = InferenceConfig()
config.display()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                93
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE         

In [16]:
# Create model object in inference mode.
model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)

print("Loading weights ", COCO_MODEL_PATH)

# Load weights trained on MS-COCO
model.load_weights(COCO_MODEL_PATH, by_name=True)

Loading weights  /home/sam/Documents/GitHub/honoursProject/PedestrianDetectionSGall/weights/mask_rcnn_coco.h5


In [17]:
# Get video file name 

vid_name = lfi.get_file_name(vid_dir)

video_path = vid_dir + vid_name
print(video_path)

/home/sam/Desktop/sam_bdd_videos/videos_for_frames/4/video/73a4bd1e-bfe891e6.mov


In [18]:
# Check if frames exist and if not break video into frames

frame_list = os.listdir(path = frames_dir) 
max = len(frame_list) 

if max == 0:
    vtf.getFrames(video_path, frames_dir)
    print("Video broken down to frames")
    frame_list = os.listdir(path = frames_dir)
    max = len(frame_list) 
else:
    print("frames already exist")

print("You can select from frame 0 to " + str(max - 1))

frames already exist
You can select from frame 0 to 1201


In [19]:
# Create collection for frame names in frames_dir

frames_list = lfi.get_file_names(frames_dir)

# Sort the frames into accending order

frames_list.sort(key=lambda f: int(re.sub('\D', '', f)))

In [25]:
frame_path_1 = os.path.join(frames_dir,frames_list[540])
frame_path_2 = os.path.join(frames_dir,frames_list[541])
frame_path_3 = os.path.join(frames_dir,frames_list[542])
frame_path_4 = os.path.join(frames_dir,frames_list[543])

frame_paths = [frame_path_1, frame_path_2, frame_path_3, frame_path_4]

frames_dict = dict()

frame_count = 0

for frame_path in frame_paths:
    
    image = cv2.imread(frame_path)
    
    # Run object detection
    results = model.detect([image], verbose=0)
    
    # Display results
    r = results[0]
    
    objects_dict = dict()
    
    objects_count = 0
    
    pedestrian_count = 0
    
    for id in r['class_ids']:
        
        if id == 1:
            
            objects_dict[objects_count] = {'pedestrian ' + str(pedestrian_count): ('Y1 = ', r['rois'][objects_count][0],
                                                                            'X1 = ', r['rois'][objects_count][1],
                                                                            'Y2 = ', r['rois'][objects_count][2],
                                                                            'X2 = ', r['rois'][objects_count][3], 
                                                                            'Accuracy = ', r['scores'][objects_count])}
            
            pedestrian_count += 1
        
        objects_count += 1
    
    frame_number = frame_path.replace("/home/sam/Desktop/sam_bdd_videos/videos_for_frames/4/frames/", "").replace(".jpg", "")
    
    frames_dict[frame_count] = {'frame ' + frame_number : objects_dict}
    
    frame_count += 1


for fd in frames_dict:
    print(frames_dict[fd])

{'frame 540': {0: {'pedestrian 0': ('Y1 = ', 200, 'X1 = ', 1014, 'Y2 = ', 458, 'X2 = ', 1134, 'Accuracy = ', 0.9991887)}, 1: {'pedestrian 1': ('Y1 = ', 174, 'X1 = ', 1141, 'Y2 = ', 455, 'X2 = ', 1259, 'Accuracy = ', 0.9989993)}, 3: {'pedestrian 2': ('Y1 = ', 142, 'X1 = ', 1241, 'Y2 = ', 423, 'X2 = ', 1280, 'Accuracy = ', 0.9822912)}}}
{'frame 541': {0: {'pedestrian 0': ('Y1 = ', 174, 'X1 = ', 1139, 'Y2 = ', 450, 'X2 = ', 1250, 'Accuracy = ', 0.9989153)}, 1: {'pedestrian 1': ('Y1 = ', 201, 'X1 = ', 1010, 'Y2 = ', 454, 'X2 = ', 1129, 'Accuracy = ', 0.9987388)}, 3: {'pedestrian 2': ('Y1 = ', 150, 'X1 = ', 1241, 'Y2 = ', 425, 'X2 = ', 1279, 'Accuracy = ', 0.9820364)}}}
{'frame 542': {0: {'pedestrian 0': ('Y1 = ', 175, 'X1 = ', 1135, 'Y2 = ', 450, 'X2 = ', 1246, 'Accuracy = ', 0.99939036)}, 1: {'pedestrian 1': ('Y1 = ', 202, 'X1 = ', 1012, 'Y2 = ', 455, 'X2 = ', 1127, 'Accuracy = ', 0.99861896)}, 2: {'pedestrian 2': ('Y1 = ', 147, 'X1 = ', 1237, 'Y2 = ', 421, 'X2 = ', 1278, 'Accuracy = ', 0

Will try and get cell above working with centroid driver and centroid tracker

In [26]:
# Initialize the parameters
confThreshold = 0.8  #Confidence threshold
nmsThreshold = 0.4   #Non-maximum suppression threshold
inpWidth = 416       #Width of network's input image
inpHeight = 416      #Height of network's input image

# initialize the video writer
writer = None

# initialize the frame dimensions (we'll set them as soon as we read
# the first frame from the video)
W = None
H = None

ct = CentroidTracker(maxDisappeared=40, maxDistance=50)
trackers = []
trackableObjects = {}

# initialize the total number of frames processed thus far, along
# with the total number of objects that have moved either up or down
totalDown = 0
totalUp = 0

In [32]:
frame_path_1 = os.path.join(frames_dir,frames_list[540])
frame_path_2 = os.path.join(frames_dir,frames_list[541])
frame_path_3 = os.path.join(frames_dir,frames_list[542])
frame_path_4 = os.path.join(frames_dir,frames_list[543])

frame_paths = [frame_path_1, frame_path_2, frame_path_3, frame_path_4]

rects = []

for frame_path in frame_paths:
    
    objects_count = 0
    
    image = cv2.imread(frame_path)
    
    # Run object detection
    results = model.detect([image], verbose=0)
    
    # Display results
    r = results[0]
    
    for id in r['class_ids']:
        
        if id == 1:
            
            if r['scores'][objects_count] > confThreshold:
                
                # x1 = Top
                top = r['rois'][objects_count][1]
                # y1 = left
                left = r['rois'][objects_count][0]
                # x2 = width
                width = r['rois'][objects_count][3]
                # y2 = height
                height = r['rois'][objects_count][2]
                
                rects.append([left, top, left + width, top + height])
                
        objects_count =+ 1
    
    objects = ct.update(rects)
    

print(objects)



OrderedDict([(0, array([ 767, 1243])), (1, array([ 803, 1368])), (2, array([ 803, 1368])), (3, array([ 799, 1364])), (4, array([ 765, 1237])), (5, array([ 765, 1237])), (6, array([ 798, 1360])), (7, array([ 765, 1239])), (8, array([ 765, 1239])), (9, array([ 797, 1356])), (10, array([ 768, 1237])), (11, array([ 768, 1237]))])


## implement this code

def counting(objects):
    frameHeight = frame.shape[0]
    frameWidth = frame.shape[1]

    global totalDown
    global totalUp

    # loop over the tracked objects
    for (objectID, centroid) in objects.items():
        # check to see if a trackable object exists for the current
        # object ID
        to = trackableObjects.get(objectID, None)

        # if there is no existing trackable object, create one
        if to is None:
            to = TrackableObject(objectID, centroid)

        # otherwise, there is a trackable object so we can utilize it
        # to determine direction
        else:
            # the difference between the y-coordinate of the *current*
            # centroid and the mean of *previous* centroids will tell
            # us in which direction the object is moving (negative for
            # 'up' and positive for 'down')
            y = [c[1] for c in to.centroids]
            direction = centroid[1] - np.mean(y)
            print(direction)
            to.centroids.append(centroid)

            # check to see if the object has been counted or not
            if not to.counted:
                # if the direction is negative (indicating the object
                # is moving up) AND the centroid is above the center
                # line, count the object

                if direction < 0 and centroid[1] in range(frameHeight//2 - 30, frameHeight//2 + 30):
                    totalUp += 1
                    to.counted = True

                # if the direction is positive (indicating the object
                # is moving down) AND the centroid is below the
                # center line, count the object
                elif direction > 0 and centroid[1] in range(frameHeight//2 - 30, frameHeight//2 + 30):
                    totalDown += 1
                    to.counted = True

        # store the trackable object in our dictionary
        trackableObjects[objectID] = to
        # draw both the ID of the object and the centroid of the
        # object on the output frame
        #text = "ID {}".format(objectID)
        #cv.putText(frame, text, (centroid[0] - 10, centroid[1] - 10),
            #cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        cv.circle(frame, (centroid[0], centroid[1]), 4, (0, 255, 0), -1)
    # construct a tuple of information we will be displaying on the
    # frame
    info = [
        ("Up", totalUp),
        ("Down", totalDown),
    ]

    # loop over the info tuples and draw them on our frame
    for (i, (k, v)) in enumerate(info):
        text = "{}: {}".format(k, v)
        cv.putText(frame, text, (10, frameHeight - ((i * 20) + 20)),
            cv.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)